## 한국환경공단 대기오염 API를 이용한 미세먼지 데이터 가져오기

In [1]:
import requests
from bs4 import BeautifulSoup
import pickle
import googlemaps
from gensim.models import Word2Vec

In [2]:
#미세먼지 및 초미세먼지 농도별 등급 판정
def grade(num) : 
    try :
        num = int(num)
        if num >=0 and num <= 30 : 
            result = "좋음"
        elif num > 30 and num <= 80 :
            result = "보통"
        elif num > 80 and num <= 150 : 
            result = "나쁨"
        else :
            result = "매우나쁨"
    except : 
        result = "-"
    return result

def nano_grade(num) : 
    try : 
        num = int(num)
        if num >=0 and num <= 15 : 
            result = "좋음"
        elif num > 15 and num <= 35 :
            result = "보통"
        elif num > 35 and num <= 75 : 
            result = "나쁨"
        else :
            result = "매우나쁨"
    except : 
        result = "-"
    return result

---------

#### 환경오염공단 API 키 불러오기

In [3]:
with open('secret/get_tm_coor_key.txt', 'rb') as f :
    get_tm_coor_key = pickle.load(f)

In [4]:
with open("secret/google_secret_key.txt", "rb") as f :
    google_secret_key = pickle.load(f)

## 1. 측정소 정보 조회
- 대기질 측정소 정보를 조회하기 위해 TM 좌표 기반의 가까운 측정소 및 측정소 목록과 측정소의 정보를 조회할 수 있음.
- 입력값 : 미세먼지 농도를 알고 싶은 위치의 주소명
- 출력값 : 위치에 대한 TM좌표
- 환경공단의 API가 인지하지 못하는 지역위치 값 입력시
> 구글 API로 위치 재조정

In [23]:
model_seoul = Word2Vec.load("model_gwangju")

In [26]:
model_seoul.wv.most_similar("치평동")

[('송하동', 0.2517915368080139),
 ('신장동', 0.2004966139793396),
 ('금호1동', 0.20010773837566376),
 ('운암3동', 0.19962099194526672),
 ('양림동', 0.18132326006889343),
 ('박호동', 0.1679231971502304),
 ('운림동', 0.16458898782730103),
 ('승촌동', 0.1280474066734314),
 ('운암2동', 0.1269875317811966),
 ('지원1동', 0.1248268187046051)]

In [ ]:
def get_tm_coor(loc) :
    
    #가독성을 위한 구글 주소 정보 속 긴 키워드들을 정리
    format_add = "formatted_address"
    add_compo = "address_components"
    ad_level = "administrative_area_level_1" # 광역시, 도
    local = "locality" # 시
    sub_1 = 'sublocality_level_1' #구
    sub_2 = 'sublocality_level_2' #동, 면
    
    korea = {
        "서울특별시" : "seoul",
        "울산광역시" : "ulsan",
        "세종광역시" : "sejong",
        "전라남도" : "junnam",
        "전라북도" : "junbuk",
        "제주특별자치도" : "jeju",
        "인천광역시" : "incheon",
        "경상남도" : "gyungnam",
        "경상북도" : "gyungbuk",
        "광주광역시" : "gwangju",
        "강원도" : "kangwon",
        "대전광역시" : "daejeon",
        "충청남도" : "chungnam",
        "충청북도" : "chungbuk",
        "경기도" : "gyunggi",
        "대구광역시" : "daegu",
        
    }
    
    #구글 API를 이용, 사용자가 알고자 하는 지역의 주소 정보 받기
    gmaps = googlemaps.Client(key = google_secret_key)
    place_info = gmaps.geocode(loc, language = 'ko')
    place_info = place_info[0][add_compo]
    
    
    try : 
        # 해당 지역의 데이터(json)에서 도(광역시), 시, 동(구) 정리
        # 에러 방지, 해당 지역의 데이터를 가져오지 않았는데 선언하는 경우
        
        for idx in range(len(place_info)) :
            if ad_level in place_info[idx]['types'] :
                do_name = place_info[idx]['long_name']

            elif local in place_info[idx]['types'] : 
                si_name = place_info[idx]['long_name']

            elif sub_1 in place_info[idx]['types'] : 
                gu_name = place_info[idx]['long_name']

            elif sub_2 in place_info[idx]['types'] : 
                dong_name = place_info[idx]['long_name']
                
        #환경공단 API가 동이름을 인식하지 못하는 경우, 주변 동을 대입해 결과물 출력(코사인 유사도 기반)
        count = 0

        while count != 1 :
            model = Word2Vec.load('model_' + korea[do_name])
            similar = model.wv.most_similar(dong_name, topn = 10)
            for idx in range(len(similar)) : 
                dong_name = similar[idx][0]
                url = "http://openapi.airkorea.or.kr/openapi/services/rest/MsrstnInfoInqireSvc/getTMStdrCrdnt?umdName="+ dong_name +"&pageNo=1&numOfRows=10&ServiceKey=" + get_tm_coor_key
                response = requests.get(url)
                dom = BeautifulSoup(response.content, "html.parser")
                count = int(dom.select("totalcount")[0].text)

    except Exception as e: 
        result = "error"


    X = dom.select("tmx")[0].text
    Y = dom.select("tmy")[0].text
    result = X, Y, do_name, dong_name

    return result

In [ ]:
get_tm_coor("대구 황금동")

In [ ]:
dong_name = "황금동"
url = "http://openapi.airkorea.or.kr/openapi/services/rest/MsrstnInfoInqireSvc/getTMStdrCrdnt?umdName="+ dong_name +"&pageNo=1&numOfRows=10&ServiceKey=" + get_tm_coor_key
response = requests.get(url)
dom = BeautifulSoup(response.content, "html.parser")
dom

----

## TM 기준좌표 조회
- 검색서비스를 사용하여 읍면동 이름을 검색조건으로 기준좌표 (TM좌표)정보를 제공하는 서비스
- 입력값으로는 TM좌표를 입력
* TM 좌표는, 위경도와 형태는 비슷하나 내용은 다른 위치 표기법

In [ ]:
with open("secret/tm_station_key.txt", "rb") as f :
    tm_station_key = pickle.load(f)

In [ ]:
def nearest_station(X, Y) : 
    url = "http://openapi.airkorea.or.kr/openapi/services/rest/MsrstnInfoInqireSvc/getNearbyMsrstnList?tmX=\
    " + X + "&tmY=" + Y +"&pageNo=1&numOfRows=10&ServiceKey=" + tm_station_key
    response = requests.get(url)
    dom = BeautifulSoup(response.content, "html.parser")
    name = dom.select("stationname")[0].text
    distance = dom.select("tm")[0].text
    return name, distance + "km"

--------

## 실시간 미세 먼지 조회
- 위치를 입력받아 실제로 미세먼지를 결과값으로 도출하는 함수

In [ ]:
def microdust_1(loc) : 
    url = "http://openapi.airkorea.or.kr/openapi/services/rest/ArpltnInforInqireSvc/getMsrstnAcctoRltmMesureDnsty?stationName="+loc+"&dataTerm=month&pageNo=1&numOfRows=10&ServiceKey="+ tm_station_key + "&ver=1.3"
    response = requests.get(url)
    dom = BeautifulSoup(response.content, "html.parser")
    date = dom.select("datatime")[0].text
    pm10 = dom.select("pm10value")[0].text
    pm25 = dom.select("pm25value")[0].text
    result = '''
    
    {}에 관측했을 때, 
    미세먼지 농도 : {}({}), 
    초 미세먼지 농도 : {}({})였습니다!
    
    선택하신 지역의 인접지역 미세먼지는 아래 버튼을 클릭하세요.
    
    '''.format(date, pm10, grade(pm10),  pm25, nano_grade(pm25))
    return result

## 인접 지역 추천
- 주소를 벡터화, 코사인 유사도를 이용한 지형적 근접 지역 추천

## 최종 함수
- 알고 싶은 위치에서 가장 가까운 관측소에서 측정한 미세먼지 및 초미세먼지 농도를 출력

In [ ]:
def get_microdust(loc) :
    try : 
        X, Y = get_tm_coor(loc)
        name, distance = nearest_station(X, Y)
        output_ls = []
        result = microdust_1(name)
    except Exception as e :
        result = "오타이거나 혹은 개발중인 기능을 요구하셨습니다."
    return result
    

In [ ]:
get_microdust("수원 영통동")

#### 클로즈 베타테스트 후 추가한 기능
- 동이름을 말했는데 국내에 중복되는 지명이 있는 경우가 있음
(흑석동 입력 시 -> 대전광역시 흑석동, 서울특별시 흑석동, 광주 광역시 흑석동 등)
- 위와 같은 경우, 리스트로 사용자에게 어느 지역인지 입력하게 함

In [ ]:
def check_detail(loc) : 
    url = "http://openapi.airkorea.or.kr/openapi/services/rest/MsrstnInfoInqireSvc/getTMStdrCrdnt?umdName="+ loc +"&pageNo=1&numOfRows=10&ServiceKey=" + get_tm_coor_key
    response = requests.get(url)
    dom = BeautifulSoup(response.content, "html.parser")
    name_ls = [name.text + " " + loc for name in dom.select("sidoname")]
    return name_ls
   